<h1><center>Laboratorio 7: Aprendizaje Supervisado 🔮</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Matías Rojas y Mauricio Araneda
- Auxiliar: Ignacio Meza D.
- Ayudante: Rodrigo Guerra

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Daniel Carmona G.
- Nombre de alumno 2: Consuelo Rojas N.


### **Link de repositorio de GitHub:** `https://github.com/DanielCG19/Evaluaciones-MDS7202`

## Temas a tratar

- Aprendizaje Supervisado
- Flujos de datos a través de `Pipelines`.

## Reglas

- Fecha de entrega: TBD
- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

## Objetivos principales del laboratorio

- Comprender el funcionamiento de clasificadores/regresores.
- Generar múltiples modelos predictivos.
- Comprender las ventajas de crear modelos en pipeline vs hacer las operaciones a mano.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre `DataFrames`.

### Importamos librerias utiles 😸

In [1]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split 

# Pre-procesamiento
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier

# Metricas de evaluación
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score

#Libreria para plotear
!pip install --upgrade plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 51.6 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.10.0
    Not uninstalling plotly at /shared-libs/python3.9/py/lib/python3.9/site-packages, outside environment /root/venv
    Can't uninstall 'plotly'. No files were found to uninstall.
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


https://www.kaggle.com/antoinekrajnc/soccer-players-statistics

# 1. Predicciones Futboleras

<p align="center">
  <img src="https://drive.google.com/uc?export=view&id=1BnO4tyh3vM2P199Ec9s3JjngQ4qQ9seP"
" width="300">
</p>


Tras el trágico despido de la mítica mascota de Maipú, Renacín decide adentrarse como consultor en el mercado futbolero, el cuál (para variar...) está cargado en especulaciones. 

Como su principal tarea será asesorar a los directivos de los clubes sobre cuál jugador comprar y cuál no, Renacín desea generar modelos predictivos que evaluén distintas características de los jugadores; todo con el fin de tomar decisiones concretas basadas en los datos.

Sin embargo, su condición de corporeo le impidió tomar la versión anterior de MDS7202, por lo que este motivo Renacín contrata a su equipo para lograr su objetivo final. Dado que aún tiene fuertes vínculos con la dirección de deportes de la municipalidad, el corporeo le entrega base de datos con las estadísticas de cada jugador para que su equipo empieze a trabajar ya con un dataset listo para ser usado.


**Los Datos**

Para este laboratorio deberán trabajar con los csv `statsplayers.csv` y `salarios.pickle`, donde deberán aplicar algoritmos de de aprendizaje supervisado (clasificación y regresión) en base a características que describen de jugadores de futbol.

Para comenzar cargue el dataset señalado y a continuación vea el reporte **`Player_Stats_Report.html`** (adjunto en la carpeta del enunciado) que describe las características principales del `DataFrame`.

In [2]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = 'Dirección donde tiene los archivos en el Drive'
except: 
    print('Ignorando conexión drive-colab')

Ignorando conexión drive-colab


In [3]:
df_players = pd.read_csv('stats_players.csv')
df_players

,Name,Nationality,National_Position,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,...,Agility,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys
0,Cristiano Ronaldo,Portugal,LS,LW,185,80,Right,32,High / Low,4,...,90,95,85,92,93,90,81,76,85,88
1,Lionel Messi,Argentina,RW,RW,170,72,Left,29,Medium / Medium,4,...,90,68,71,85,95,88,89,90,74,85
2,Neymar,Brazil,LW,LW,174,68,Right,25,High / Medium,5,...,96,61,62,78,89,77,79,84,81,83
3,Luis Suárez,Uruguay,LS,ST,182,85,Right,30,High / Medium,4,...,86,69,77,87,94,86,86,84,85,88
4,Manuel Neuer,Germany,GK,GK,193,92,Right,31,Medium / Medium,4,...,52,78,25,25,13,16,14,11,47,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17583,Adam Dunbar,Republic of Ireland,NaN,Sub,183,82,Right,19,Medium / Medium,1,...,27,56,14,16,13,13,11,13,15,12
17584,Dylan McGoey,Republic of Ireland,NaN,Sub,185,80,Right,19,Medium / Medium,2,...,28,53,12,17,12,11,12,13,16,12
17585,Tommy Ouldridge,England,NaN,Res,173,61,Right,18,High / Medium,2,...,54,61,41,44,28,42,35,36,42,37
17586,Mark Foden,Scotland,NaN,Sub,180,80,Right,21,Medium / Medium,3,...,34,48,15,23,14,12,13,12,24,12


## 1.1 Predicción de Seleccionados Nacionales 

Como primera tarea, Renacín, intrigado por la posibilidad de saber qué tan reconocido es un jugador, le consulta a su equipo si es posible predecir si un jugador será o no seleccionado nacional a partir de sus estadísticas en el juego. 

<p align="center">
  <img src="https://thumbs.gfycat.com/ThornyFrankFoxhound-size_restricted.gif" width="500">
</p>

### 1.1.1 Generación de Labels para la Clasificación [Sin Puntaje]


Primero comience generando las labels para la clasificación. Para esto, trabaje sobre el atributo ```National_Position``` suponiendo que los valores nulos son jugadores no seleccionados para representar a su país. 

Hecho esto, ¿Cuantos ejemplos por cada clase se tienen? Comente lo que observa.



**Respuesta:**

In [4]:
#NC de No Clasificado

df_players.National_Position.fillna('NC',inplace=True)

df_players['National_Position'].value_counts(normalize=True) * 100
#print(len(df_players))


NC     93.887878
Sub     3.161246
LCB     0.272913
GK      0.267228
RCB     0.261542
LB      0.221742
RB      0.216056
RM      0.193314
LM      0.181942
ST      0.170571
RCM     0.142142
LCM     0.142142
CAM     0.108028
LDM     0.108028
RS      0.102343
RDM     0.102343
LS      0.102343
CM      0.051171
CB      0.051171
CDM     0.051171
RW      0.039800
LW      0.039800
LAM     0.022743
RWB     0.022743
LWB     0.022743
RAM     0.022743
LF      0.017057
RF      0.017057
Name: National_Position, dtype: float64

### 1.1.2 Camino a la clasificación [1 punto]

<p align="center">
  <img src="https://c.tenor.com/SoXZJHWnhKAAAAAC/pitbull-medel.gif" width="500">
</p>

Para preprocesar el dataset, genere un `ColumnTransformer` en donde especifique las transformaciones que hay que realizar para cada columna (por ejemplo StandarScaler, MinMaxScaler, OneHotEncoder, etc...) para que puedan ser utilizadas correctamente por el modelo predictivo y guardelo en algúna variable.

Luego, comente y justifique las transformaciones elegidas sobre cada una de las variables (para esto utilice el material ```Player_Stats_Report.html``` que viene en el zip del lab), al igual que las transformaciones aplicadas.

Hecho lo anterior, **defina al menos 3 pipelines para la clasificación**, en donde utilice el mismo ColumnTransformer definido anteriormente, pero que varie entre cada pipeline los clasificadores. 

Para seleccionar los clasificadores más adecuados, utilice la siguiente guía:

<p align="center">
  <img src="https://scikit-learn.org/stable/_static/ml_map.png" width="500">
</p>

<br/>

Con ella, comente y justifique cada una de las decisiones tomadas al momento de desarrollar su pipeline.

**Nota:** Si tiene problemas al utilizar OneHotEncoder puede utilizar el parámetro handle_unknown='ignore'. Esto hace que en la codificación se omitan las categorias que no aparecen en el entrenamiento. Pregunta dudosa (no tiene puntaje), ¿esto tiene sentido a nivel de modelos?.

**To-Do:**

- [ ] Genere un ```ColumnTransformer``` enfocado en preprocesar los datos.
- [ ] Indicar y Justificar que preprocesamiento utiliza sobre cada columna.
- [ ] Crear 3 ``pipelines`` con diferentes clasificadores.
- [ ] Para seleccionar los clasificadores base sus decisiones en la siguiente [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html)
- [ ] No entrenar los ``pipelines`` aún.

**Nota:** No es necesario entrenar los clasificadores aún.

**Respuesta:**

In [5]:
#df_players.columns
# df_players.Work_Rate.unique()
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

Para la selección de transformadores para las columnas se utilizó el siguiente criterio:

- Números del tipo float, se utiliza StandarScaler(), de manera de estadarizar estos datos. Se utilizó para cosas como Age, Height y Weight

- Las hábilidades, que tenían una puntación de 0 a 5, se transformaron con un MinMaxScaler(), dejando así todos en una misma escala. Esto se utilizó para columnas como Weak foot, Skill Moves, Crossing, etc.

- Para las posiciones o preferencias representadas con caracteres, se utilizó OneHotEncoder(), de manera de asignar un vector significativo a las distintas opciones.

In [6]:
ct = ColumnTransformer(
    [("StandarScaler", StandardScaler(), ["Age",'Height','Weight']),
    ("MinMax", MinMaxScaler(), ['Weak_foot','Skill_Moves', 'Ball_Control', 'Dribbling', 'Marking', 
        'Sliding_Tackle', 'Standing_Tackle', 'Aggression', 'Reactions', 'Interceptions', 'Vision',
       'Composure', 'Crossing', 'Short_Pass', 'Long_Pass', 'Acceleration', 'Speed', 'Stamina', 
       'Strength', 'Balance', 'Agility', 'Jumping', 'Heading', 'Shot_Power', 'Finishing', 'Long_Shots', 
       'Curve', 'Freekick_Accuracy', 'Penalties', 'Volleys']),
    ("OneHot",OneHotEncoder(),["Club_Position","Preffered_Foot","Work_Rate"])
    #("OneHot",OneHotEncoder(),["National_Position","Club_Position","Preffered_Foot","Work_Rate"])
    ], remainder="passthrough")

steps = [("Preprocesamiento", ct)]
pipe = Pipeline(steps)

In [7]:
scaled_df_players = pd.DataFrame(
    pipe.fit_transform(df_players)#, columns=df_players.columns
)

In [8]:
display(scaled_df_players)

,0,1,2,3,4,5,6,7,8,9,...,67,68,69,70,71,72,73,74,75,76
0,1.397344,0.583453,0.688144,0.75,1.0,0.977778,0.946237,0.213483,0.2,0.314607,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Cristiano Ronaldo,Portugal,LS
1,0.75633,-1.663748,-0.471654,0.75,0.75,1.0,1.0,0.11236,0.233333,0.280899,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,Lionel Messi,Argentina,RW
2,-0.098356,-1.064495,-1.051553,1.0,1.0,1.0,0.989247,0.202247,0.311111,0.235955,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,Neymar,Brazil,LW
3,0.970001,0.134013,1.413018,0.75,0.75,0.955556,0.88172,0.303371,0.366667,0.47191,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,Luis Suárez,Uruguay,LS
4,1.183672,1.78196,2.427841,0.75,0.0,0.477778,0.27957,0.078652,0.066667,0.078652,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,Manuel Neuer,Germany,GK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17583,-1.380384,0.283826,0.978093,0.0,0.0,0.077778,0.075269,0.101124,0.077778,0.11236,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,Adam Dunbar,Republic of Ireland,NC
17584,-1.380384,0.583453,0.688144,0.25,0.0,0.088889,0.075269,0.089888,0.077778,0.089888,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,Dylan McGoey,Republic of Ireland,NC
17585,-1.594055,-1.214308,-2.066377,0.25,0.25,0.433333,0.376344,0.41573,0.266667,0.359551,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,Tommy Ouldridge,England,NC
17586,-0.953041,-0.165614,0.688144,0.5,0.0,0.133333,0.096774,0.089888,0.066667,0.123596,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,Mark Foden,Scotland,NC


In [9]:
steps_svm = [("Preprocesamiento", ct), ('linearsvc', LinearSVC(random_state=0, tol=1e-05))]
steps_nb = [("Preprocesamiento", ct), ('NaiveBayes', GaussianNB())]
steps_kn = [("Preprocesamiento", ct), ('KNeighbors', KNeighborsClassifier(n_neighbors=3))]

pipe_svm = Pipeline(steps_svm)
pipe_nb = Pipeline(steps_nb)
pipe_kn = Pipeline(steps_kn)

### 1.1.3 Entrenemos los pipelines [1 punto]

<p align="center">
  <img src="https://i.gifer.com/7Cc8.gif" width="350">
</p>


Ahora, entrene los `pipeline` generados en los pasos anteriores. Para esto, primero separe los datos de entrenamiento en un conjunto de entrenamiento y de prueba  (la proporción queda a su juicio).

En este paso, seleccione los ejemplos de forma aleatoria e intente mantener la distribución original de labels de cada clase en los conjuntos de prueba/entrenamiento. (vea la documentación de `train_test_split`).


Luego, entrene los pipelines

Una vez entrenado su modelo, evalue su rendimiento a través de diferentes métricas, comentando que significa cada uno de los valores obtenidos. Puede usar la función `classification_report` para corroborar sus resultados.

- ¿Qué implican los valores de accuracy, precisión y recall de la clase positiva (la que indica que un jugador es seleccionado nacional)?
- ¿Podrían mejorarse los resultados?, ¿Cómo?
- ¿Influye la cantidad de ejemplos por clase?

**To-Do**:

- [ ] Separar el conjunto de datos en entrenamiento y de prueba.
- [ ] Entrenar los `pipelines`.
- [ ] Utilizar las métricas para evaluar los modelos generados.


**Respuesta:**

In [10]:
df_players['Seleccionado'] = df_players['National_Position'].apply(lambda x: 0 if x == 'NC' else 1)
display(df_players)

,Name,Nationality,National_Position,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,...,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys,Seleccionado
0,Cristiano Ronaldo,Portugal,LS,LW,185,80,Right,32,High / Low,4,...,95,85,92,93,90,81,76,85,88,1
1,Lionel Messi,Argentina,RW,RW,170,72,Left,29,Medium / Medium,4,...,68,71,85,95,88,89,90,74,85,1
2,Neymar,Brazil,LW,LW,174,68,Right,25,High / Medium,5,...,61,62,78,89,77,79,84,81,83,1
3,Luis Suárez,Uruguay,LS,ST,182,85,Right,30,High / Medium,4,...,69,77,87,94,86,86,84,85,88,1
4,Manuel Neuer,Germany,GK,GK,193,92,Right,31,Medium / Medium,4,...,78,25,25,13,16,14,11,47,11,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17583,Adam Dunbar,Republic of Ireland,NC,Sub,183,82,Right,19,Medium / Medium,1,...,56,14,16,13,13,11,13,15,12,0
17584,Dylan McGoey,Republic of Ireland,NC,Sub,185,80,Right,19,Medium / Medium,2,...,53,12,17,12,11,12,13,16,12,0
17585,Tommy Ouldridge,England,NC,Res,173,61,Right,18,High / Medium,2,...,61,41,44,28,42,35,36,42,37,0
17586,Mark Foden,Scotland,NC,Sub,180,80,Right,21,Medium / Medium,3,...,48,15,23,14,12,13,12,24,12,0


In [11]:
X = df_players.drop(['Name','Nationality','National_Position'], axis=1) # probar con one hot par nacionalidad
y = df_players['Seleccionado']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

print(np.unique(y_train, return_counts=True))
print(np.unique(y_test, return_counts=True))

(array([0, 1]), array([13210,   860]))
(array([0, 1]), array([3303,  215]))


In [12]:
pipe_svm.fit(X_train, y_train)
y_pred_svm = pipe_svm.predict(X_test)
print(classification_report(y_test, y_pred_svm))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3303
           1       1.00      1.00      1.00       215

    accuracy                           1.00      3518
   macro avg       1.00      1.00      1.00      3518
weighted avg       1.00      1.00      1.00      3518



In [13]:
pipe_kn.fit(X_train, y_train)
y_pred_kn = pipe_kn.predict(X_test)
print(classification_report(y_test, y_pred_kn))

              precision    recall  f1-score   support

           0       0.97      1.00      0.99      3303
           1       0.99      0.57      0.73       215

    accuracy                           0.97      3518
   macro avg       0.98      0.79      0.86      3518
weighted avg       0.97      0.97      0.97      3518



In [14]:
pipe_nb.fit(X_train, y_train)
y_pred_nb = pipe_nb.predict(X_test)
print(classification_report(y_test, y_pred_nb))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3303
           1       1.00      1.00      1.00       215

    accuracy                           1.00      3518
   macro avg       1.00      1.00      1.00      3518
weighted avg       1.00      1.00      1.00      3518



**Respuesta**:

```
Las distintas métricas presentan el nivel de desempeño que tiene el modelo de clasificación, ya sea para detectar buenas o malas clasificaciones.

El accuracy es la medida que ve la cantidad de aciertos, es decir, predicciones correctas en relación al total de las predicciones hechas. Esto significa que entre más cercano a 1 el valor, es más aciertos tiene el clasificador.

La presición corresponde a la cantidad de verdaderos positivos sobre la cantidad total de verdaderos positivos y falsos positivos. Es decir, la razón entre los aciertos positivos y los aciertos generales.

Finalmente el recall representa la razón entre los verdaderos positivos y los positivos totales en el problema.

En especifico, 1 corresponde a la clase de seleccionados. Los clasificadores de SVM y NB, dan clasificaciones perfectas. Esto significa que todos los jugadores predichos que serán seleccionados, es porque lo serán.

Para el caso de KN, hay una presición de 0.97, cercana a 1, lo que significa que todos los jugadores clasificados como seleccionados, tienen una buena certeza alta de que esa etiqueta les corresponda. No así el recall, que tiene un puntaje de 0.52, significando que hay probabilidades de que a jugadores a los que se predijo como no seleccionados, si lo sean.

Es bastante díficil mejorar estos resultados, pero una posible mejora, sobre todo para el recall de KN podría realizarse con un equilibrio en las clases.

En este caso, no se nota mucho la influencia de la cantidad de datos por clase, pero en general si es necesario tener una cantidad homogenea de datos en las clases, de manera de poder cubrir todos los casos y así evitar que el sistema aprenda sesgos por falta de datos.
```

## 1.2 Predicción de posiciones de jugadores [2 puntos]

<p align="center">
  <img src="https://pbs.twimg.com/media/E1rfA1aWEAYU6Ny.jpg" width="500">
</p>

En una nueva jornada de desmesuradas transacciones deportivas, Renacín escuchó a sus colegas discutir acerca de que el precio de cada jugador depende en gran medida de la posición en la cancha en la que juega. Y además, que hay bastantes jugadores nuevos que no tienen muy claro en que posición verdaderamente brillarían, por lo que actualmente puede que actualmente estén jugando en posiciones sub-optimas. 

Viendo que los resultados del primer análisis no son tan esperanzadores, el corporeo los comanda a cambiar su tarea: ahora, les solicita que construyan un clasificador enfocado en predecir la mejor posición de los jugadores en la cancha según sus características. 

Para lograr esto, primero, les pide que etiqueten de la siguiente manera los valores que aparecen en el atributo ```Club_Position```, pidiendo que agrupen los valores en los siguientes grupos:

**Nota**:  Renacín les recalca que **no deben utilizar los valores ```Sub``` y ```Res``` de esta columna**.

```python
ataque = ['ST', 'CF'] 
central_ataque = ['RW', 'CAM', 'LW'] 
central = ['RM', 'CM', 'LM'] 
central_defensa = ['RWB', 'CDM', 'LWB']
defensa = ['RB', 'CB', 'LB']
arquero = ['GK']
```

Cabe señalar que al igual como lo realizado con la clasificación binaria, deberá justificar en base a la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) la elección del clasificador y se deben comentar los resultados obtenidos en la clasificación.

**Nota:** Clasifique solamente con las clases señaladas, si observa mas clases eliminelas de la clasficación.

**To-Do**:

- [ ] Aplique las etiquetas descritas anteriormente en cada uno de los valores señalados en esta sección.
- [ ] Cuente cuantos por clase quedan.
- [ ] Entrene el nuevo pipeline y ejecute una evaluación de este.  
- [ ] Comente los resultados obtenidos.

**Respuesta:**

In [15]:
### Código aquí ### 
values = ['ST', 'CF', 'RW', 'CAM', 'LW', 'RM', 'CM', 'LM', 'RWB', 'CDM', 'LWB', 'RB', 'CB', 'LB', 'GK']

df_players2 = df_players.copy()
df_players2 = df_players2[df_players2['Club_Position'].isin(values)]

dict = {"ST" : 'ataque', "CF" : 'ataque',
        "RW": 'central ataque', "CAM" : 'central ataque', "LW": 'central ataque',
        "RM": 'central', "CM" : 'central', "LM": 'central',
        "RWB": 'central defensa', "CDM" : 'central defensa', "LWB": 'central defensa',
        "RB": 'defensa', "CB" : 'defensa', "LB": 'defensa',
        "GK": 'arquero'}
df_players2 = df_players2.replace({'Club_Position': dict})

df_players2.Club_Position.value_counts()

defensa            1180
central             907
arquero             632
central ataque      581
ataque              430
central defensa     209
Name: Club_Position, dtype: int64

In [16]:
X = df_players2.drop(['Name','Nationality','National_Position','Seleccionado'], axis=1) # probar con one hot par nacionalidad
y = df_players2['Club_Position']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

print(np.unique(y_train, return_counts=True))
print(np.unique(y_test, return_counts=True))

(array(['arquero', 'ataque', 'central', 'central ataque',
       'central defensa', 'defensa'], dtype=object), array([506, 344, 725, 465, 167, 944]))
(array(['arquero', 'ataque', 'central', 'central ataque',
       'central defensa', 'defensa'], dtype=object), array([126,  86, 182, 116,  42, 236]))


In [17]:
pipe_svm.fit(X_train, y_train)
y_pred_svm = pipe_svm.predict(X_test)
print(classification_report(y_test, y_pred_svm))

                 precision    recall  f1-score   support

        arquero       1.00      1.00      1.00       126
         ataque       1.00      1.00      1.00        86
        central       1.00      1.00      1.00       182
 central ataque       1.00      1.00      1.00       116
central defensa       1.00      1.00      1.00        42
        defensa       1.00      1.00      1.00       236

       accuracy                           1.00       788
      macro avg       1.00      1.00      1.00       788
   weighted avg       1.00      1.00      1.00       788



In [18]:
pipe_kn.fit(X_train, y_train)
y_pred_kn = pipe_kn.predict(X_test)
print(classification_report(y_test, y_pred_kn))

                 precision    recall  f1-score   support

        arquero       0.99      1.00      1.00       126
         ataque       1.00      0.97      0.98        86
        central       0.98      1.00      0.99       182
 central ataque       0.99      1.00      1.00       116
central defensa       1.00      0.90      0.95        42
        defensa       1.00      1.00      1.00       236

       accuracy                           0.99       788
      macro avg       0.99      0.98      0.99       788
   weighted avg       0.99      0.99      0.99       788



In [19]:
pipe_nb.fit(X_train, y_train)
y_pred_nb = pipe_nb.predict(X_test)
print(classification_report(y_test, y_pred_nb))

                 precision    recall  f1-score   support

        arquero       1.00      1.00      1.00       126
         ataque       1.00      1.00      1.00        86
        central       1.00      1.00      1.00       182
 central ataque       1.00      1.00      1.00       116
central defensa       1.00      1.00      1.00        42
        defensa       1.00      1.00      1.00       236

       accuracy                           1.00       788
      macro avg       1.00      1.00      1.00       788
   weighted avg       1.00      1.00      1.00       788



**Respuesta**:

```
Se puede notar, que nuevamente las clasificaciones son casi perfectas. Esto se debe a que los datos estan bien estandarizados y a difenrecia de la clasificaicón binaria, acá hay una distribución más "homogenea" en las clases. Lo que hace que hayan menos sesgos, en general.

La única manera de mejorar el sistema, sería equilibrando aún más las clases, de manera que el clasificador KN cuente con toda la información necesaria para poder encontrar los vecinos más cercanos correctos.
```

## 1.3 Predicción de Sueldos [2 puntos]

<p align="center">
  <img src="https://media.giphy.com/media/3oEjHWtHAFcOKS4iA0/giphy.gif" width="460">
</p>

Queriendo ahondar aún más en el mercado del balompíe, Renacin, logra obtener (de una manera no muy formal) los sueldos de múltiples futbolistas y los guarda en el archivo ```sueldos.csv```. Con ellos les solicita que generen un regresor que les permita predecir el sueldo de los futbolistas en base a las características de los pichichis, esto, debido a su motivación por invertir y/o realizar especulación sobre los sueldos de jugadores.

Renacin es claro señalando que deben seguir utilizando la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) y comenten cada uno de los pasos realizados, para obtener su regresión lineal. Señalándoles que no aceptara un $R^2$ inferior a 0.35 para el modelo solicitado.

Para esta parte usted tiene total libertad en la generación del regresor, la unica exigencia es que utilice un pipeline para generar la regresión y utilice la metrica $R^2$ para medir el rendimiento de esta.

**To-Do**:

- [ ] Explique en que consiste la métrica $R^2$
- [ ] Generar un pipeline para la regresión.
- [ ] Obtener un regresor con un $R^2$ superior a  $0.35$.
- [ ] Comente sus resultados y si es posible mejorar los resultados obtenidos. ¿Se necesitarían más datos o otros tipos de características o una combinación de ambos?

**Respuesta**

La métrica $R^2$ o también llamada coeficiente de determinación, corresponde a una proporción entre lo bueno que es el nuestro modelo en comparación a uno naïve o que siempre retorne la media. Entre más cercano a 1 el valor del error, más certera es el modelo aplicado. Un error cercano a 0, indica que el modelo tienede a acertar lo mismo que uno ingenuo y un error negativo, nos dice que nuestro modelo es peor que retornar siempre la media.

In [20]:
### Código aquí ###
##Se lee el dataframe con los sueldos y se ven las variables.
df_sueldos = pd.read_csv('sueldos.csv')
df_sueldos.head()

,Unnamed: 0,Player,Weekly Salary
0,1528,Lionel Messi,1538905.0
1,1006,Cristiano Ronaldo,1248536.0
2,1529,Antoine Griezmann,997032.0
3,2004,Neymar,797726.0
4,1530,Gareth Bale,607706.0


Notar que hay 3 columnas, además del index. Estas corresponden al ID del jugador, el nombre y su sueldo. Para hacer el regresor, se va a necesitar añadir la información de los sueldos a la de las caracteristicas.

In [21]:
df_salaries = df_players2.copy()

Luego, se hace una copia del dataframe original para trabajar con la copia y así no alterar los valores originales y poder volver a recurrir a ellos, de ser necesario.

Luego, a esta copia se le hace un merge con el data de los sueldos en las columnas de "Name" y "Player" correspondiente. Luego se eliminan todas las columnas que no pueden ser cambiadas por int o floats, esto se debe a que los regresores no pueden trabajar con strings.

In [22]:
df_salaries = df_salaries.merge(df_sueldos, left_on='Name', right_on='Player')
df_salaries.rename({'Unnamed: 0': 'Id'},inplace= True)
df_salaries = df_salaries.drop(['Name','Player','Nationality','National_Position'], axis = 1)
df_salaries.head()


,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,Skill_Moves,Ball_Control,Dribbling,...,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys,Seleccionado,Unnamed: 0,Weekly Salary
0,central ataque,185,80,Right,32,High / Low,4,5,93,92,...,92,93,90,81,76,85,88,1,1006,1248536.0
1,central ataque,170,72,Left,29,Medium / Medium,4,4,95,97,...,85,95,88,89,90,74,85,1,1528,1538905.0
2,central ataque,174,68,Right,25,High / Medium,5,5,95,96,...,78,89,77,79,84,81,83,1,2004,797726.0
3,ataque,182,85,Right,30,High / Medium,4,4,91,86,...,87,94,86,86,84,85,88,1,1531,508923.0
4,arquero,193,92,Right,31,Medium / Medium,4,1,48,30,...,25,13,16,14,11,47,11,1,498,326233.0


Una vez creado el dataframe, este se divide en dos. X,y: que corresponde al set de caracteristicas y el target. En este caso, el target del regresor corresponde al sueldo semanal de un jugador.

Una vez separadas las caracteristicas de los datos, se hacen los datos de entrenamiento y testeo. Además se elimina la columna "Unnamed: 0", dado que es un Id de los jugadores.

In [41]:
y = df_salaries['Weekly Salary']
X = df_salaries.drop(['Weekly Salary', 'Unnamed: 0'], axis = 1)

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state=7)


Luego, se generan los pipeline para los regresores. Una vez se tienen los pipelines, se entrenan con los datos y luego de procede a predecir los valores del dataset de test.

Para ver el score, se utiliza r2_score de sklearn.metrics.

In [43]:
from sklearn.svm import SVR
from sklearn.metrics import r2_score

steps_svr = [("Preprocesamiento", ct),("SVR",SVR(kernel = 'linear'))]
pipe_svr = Pipeline(steps_svr)

pipe_svr.fit(X_train,y_train)
y_pred_svr = pipe_svr.predict(X_test)
score_svr = r2_score(y_pred_svr,y_test)

print(f'R2 de regresor SVR:\t {score_svr}')

R2 de regresor SVR:	 -294470.13155361457


In [44]:
from sklearn.linear_model import Ridge

steps_rd = [("Preprocesamiento", ct),("Ridge",Ridge(alpha = 1.0,max_iter=10000))]
pipe_rd = Pipeline(steps_rd)

pipe_rd.fit(X_train,y_train)
y_pred_rd = pipe_rd.predict(X_test)

print(f'R2 de regresor Ridge:\t {r2_score(y_pred_rd,y_test)}')

R2 de regresor Ridge:	 0.05257147082694025


In [35]:
from sklearn.ensemble import AdaBoostRegressor

regr = AdaBoostRegressor(random_state=0, n_estimators=100)
steps_ada = [("Preprocesamiento", ct), ('AdaBoost', regr)]
pipe_ada = Pipeline(steps_ada)

pipe_ada.fit(X_train,y_train)
y_pred_ada = pipe_ada.predict(X_test)
score_ada = r2_score(y_pred_ada,y_test)

print(f'R2 de regresor AdaBoost:\t {score_ada}')

R2 de regresor AdaBoost:	 -1.531774082367015


In [37]:
from sklearn.ensemble import GradientBoostingRegressor

grad = GradientBoostingRegressor(
    n_estimators=100, learning_rate=0.1, max_depth=1, random_state=0,
    loss='squared_error' )
steps_gradient = [("Preprocesamiento", ct), ('GradientBoost', grad)]
pipe_gradient = Pipeline(steps_gradient)

pipe_gradient.fit(X_train,y_train)
y_pred_gradient = pipe_gradient.predict(X_test)
score_gradient = r2_score(y_pred_gradient,y_test)

print(f'R2 de regresor GradientBoosting:\t {score_gradient}')

R2 de regresor GradientBoosting:	 0.5750942312384771


**Respuesta**:

```
Se hicieron 4 regresores, de los cuales solo 1 pasó la mínima puesta por el señor Renacín.

Primero, siguiendo la guía se utilizaron los modelos líneales expuestos ahí. Podemos notar que el SVR tiene un score R2 muy negativo. Esto significa que las regresiones hechas, dieron todas muy distintas al resultado esperado y que una mejor aproximación al problema habría sido ofrecerle a los jugadores el sueldo medio. Esto se repite con el regresor AdaBoosting

Por otro lado, Ridge tiende a 0, lo que significa que el regresor tiende a ofrecer la media de los sueldos a los jugadores.

AdaBoosting falla porque al tener una gran cantidad de opciones y este al tener incrementos de aprendizaje "lento", necesita de una gran cantidad de iteraciones para lograr el objetivo. Este regresor tiende a ser mejor en problemas con soluciones finitas y limitadas a un conjunto y no abiertas, como el actual. Esto mismo se repite para el caso de SVR.

Para Ridge, es un regresor lineal, lo que significa que cualquier tipo de variable que no responda a un modelo lineal, este regresor no podra adecuarse a ella lo suficiente para representarla, por ello tampoco sirve como es de esperarse.

Finalmente, tenemos el GradientBoosting, que fue el único que sobre paso el mínimo pedido. Este regresor, es capaz de leer funciones no lineales, esto explica el porque funciona en comparación a los otros.
```

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media2.giphy.com/media/l1J3CbFgn5o7DGRuE/giphy.gif">
</p>

<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=5e9a7b18-a9f1-472c-a18e-016f14247eca' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>